In [70]:
import numpy as np
import pandas as pd
from astropy.io import fits
import glob
from collections import defaultdict
import sqlite3 as lite

In [4]:
# Read in the field information
root = "/home/esmee/Documents/DBDM/DDM2017-master/FinalProject/" #"/disks/strw9/stoop/DBDM/DDM2017-master/FinalProject/Q1/Tables/"
"""
field_info = pd.read_csv(root + 'file_info_for_problem.csv')
field_info.insert(column="StarID", loc=4, value = 0)
print field_info
"""

'\nfield_info = pd.read_csv(root + \'file_info_for_problem.csv\')\nfield_info.insert(column="StarID", loc=4, value = 0)\nprint field_info\n'

In [5]:
# Read in the different fits files
fnames = glob.glob(root + '*.fits')

#Find the number of unique stars
starIDs = []
for i, f in enumerate(fnames):
    hdu = fits.open(f)
    starIDs.append([dat for dat in hdu[1].data['StarID']])

nstars = len(np.unique(starIDs))
starCols = np.array(['ID', 'StarID', 'Ra', 'Dec', 'Flux1_Z', 'Flux1_Y', 'Flux1_J', 'Flux1_H', 'Flux1_Ks-E001',
       'Flux1_Ks-E002', 'Flux1_Ks-E003', 'Flux2_Z', 'Flux2_Y', 'Flux2_J',
       'Flux2_H', 'Flux2_Ks-E001', 'Flux2_Ks-E002', 'Flux2_Ks-E003',
       'Flux3_Z', 'Flux3_Y', 'Flux3_J', 'Flux3_H', 'Flux3_Ks-E001',
       'Flux3_Ks-E002', 'Flux3_Ks-E003', 'dFlux1_Z', 'dFlux1_Y',
       'dFlux1_J', 'dFlux1_H', 'dFlux1_Ks-E001', 'dFlux1_Ks-E002',
       'dFlux1_Ks-E003', 'dFlux2_Z', 'dFlux2_Y', 'dFlux2_J', 'dFlux2_H',
       'dFlux2_Ks-E001', 'dFlux2_Ks-E002', 'dFlux2_Ks-E003', 'dFlux3_Z',
       'dFlux3_Y', 'dFlux3_J', 'dFlux3_H', 'dFlux3_Ks-E001',
       'dFlux3_Ks-E002', 'dFlux3_Ks-E003', 'Mag1_Z', 'Mag1_Y', 'Mag1_J',
       'Mag1_H', 'Mag1_Ks-E001', 'Mag1_Ks-E002', 'Mag1_Ks-E003', 'Mag2_Z',
       'Mag2_Y', 'Mag2_J', 'Mag2_H', 'Mag2_Ks-E001', 'Mag2_Ks-E002',
       'Mag2_Ks-E003', 'Mag3_Z', 'Mag3_Y', 'Mag3_J', 'Mag3_H',
       'Mag3_Ks-E001', 'Mag3_Ks-E002', 'Mag3_Ks-E003', 'dMag1_Z',
       'dMag1_Y', 'dMag1_J', 'dMag1_H', 'dMag1_Ks-E001', 'dMag1_Ks-E002',
       'dMag1_Ks-E003', 'dMag2_Z', 'dMag2_Y', 'dMag2_J', 'dMag2_H',
       'dMag2_Ks-E001', 'dMag2_Ks-E002', 'dMag2_Ks-E003', 'dMag3_Z',
       'dMag3_Y', 'dMag3_J', 'dMag3_H', 'dMag3_Ks-E001', 'dMag3_Ks-E002',
       'dMag3_Ks-E003']) 

fmcols = ['Flux', 'dFlux', 'Mag', 'dMag']
uniq_IDs = np.unique(starIDs)
nstars = len(np.unique(starIDs))
ncols = 5 + 83
print "Table is {0} x {1}".format(nstars, ncols)
stable = np.zeros((nstars, ncols))
starTable = pd.DataFrame(data=stable, columns=starCols)

Table is 30000 x 88


In [10]:
len(fnames[0].replace(root, ''))

14


# The database tables

**FieldTable**

ID | FieldID | Filename | Filter | Exp time | MJD | StarIDs |


**StarTable**

ID  | StarID  | Ra  | Dec  | Flux1_Z  | dFlux1_Z  | ..  | Flux3_Ks  | dFlux3_Ks  | Mag1_Z  | dMag1_Z  | ..  | Mag3_Ks  | dMag3_Ks  | Var_Ks |  



In [6]:
"""
IDs_seen = []
fmcols = ['Flux', 'dFlux', 'Mag', 'dMag']

for f in fnames:
    print "Started with file {0}".format(f.replace(root, ''))
    
    hdu = fits.open(f)
    f = f.replace(root, '')
    sID = hdu[1].data['StarID']
    ind = [np.where(uniq_IDs == i)[0][0] for i in sID]
    filt = f[8:-5]
    
    #Send star IDs to the field_info
    fID = f[6] #field
    if 'Ks' in filt:
        filt2 = 'Ks'
        name = np.sort(np.array(field_info["Filename"][(field_info["FieldID"] == int(fID)) & (field_info["Filter"] == filt2)]))
        name = name[int(filt[-1])-1]
    else:
        name = np.sort(np.array(field_info["Filename"][(field_info["FieldID"] == int(fID)) & (field_info["Filter"] == filt)]))
        name = name[0]
    field_info["StarID"][np.where(field_info["Filename"] == name)[0][0]] = np.array_str(sID)
    
    seen = [ID in IDs_seen for ID in sID]

    # Is iD star already seen?
    if not seen[0]:
        starTable['ID'][ind] = ind
        # Input the StarID, Ra and Dec
        for c in starCols[1:4]:
            starTable[c][ind] = hdu[1].data[c]

        [IDs_seen.append(ID) for ID in sID]
        
        # Input the flux and magnitude of this data
        for fm in fmcols:
            for t in range(1, 4):
                starTable[fm + str(t) +'_' + filt][ind] = hdu[1].data[fm + str(t)]

    # Go through all other flux, mag and filters
    else:
        for fm in fmcols:
            for t in range(1, 4):
                starTable[fm + str(t) +'_' + filt][ind] = hdu[1].data[fm + str(t)]
    

    print "Finished"

# write tables to csv
starTable.to_csv(root+'starTable.csv')
field_info.to_csv(root+'fieldTable.csv')
"""

'\nIDs_seen = []\nfmcols = [\'Flux\', \'dFlux\', \'Mag\', \'dMag\']\n\nfor f in fnames:\n    print "Started with file {0}".format(f.replace(root, \'\'))\n    \n    hdu = fits.open(f)\n    f = f.replace(root, \'\')\n    sID = hdu[1].data[\'StarID\']\n    ind = [np.where(uniq_IDs == i)[0][0] for i in sID]\n    filt = f[8:-5]\n    \n    #Send star IDs to the field_info\n    fID = f[6] #field\n    if \'Ks\' in filt:\n        filt2 = \'Ks\'\n        name = np.sort(np.array(field_info["Filename"][(field_info["FieldID"] == int(fID)) & (field_info["Filter"] == filt2)]))\n        name = name[int(filt[-1])-1]\n    else:\n        name = np.sort(np.array(field_info["Filename"][(field_info["FieldID"] == int(fID)) & (field_info["Filter"] == filt)]))\n        name = name[0]\n    field_info["StarID"][np.where(field_info["Filename"] == name)[0][0]] = np.array_str(sID)\n    \n    seen = [ID in IDs_seen for ID in sID]\n\n    # Is iD star already seen?\n    if not seen[0]:\n        starTable[\'ID\'][ind] 

In [76]:
# Read the Tables
FieldTable = pd.read_csv(root+ "fieldTable.csv")
StarTable = pd.read_csv(root+ "starTable.csv")

FieldTable.drop('Unnamed: 0', axis=1, inplace=True)
StarTable.drop('Unnamed: 0', axis=1, inplace=True)

In [77]:
# Make the database
con = lite.connect(root+"VVV.db")
StarTable.to_sql('StarTable', con, if_exists = 'replace')
FieldTable.to_sql('FieldTable', con, if_exists = 'replace')

# The Queries

**R1:** Find all images observed between MJD = 56800 and MJD = 57300 and give me the number of stars detected with S/N > 5 in each image.

**R2:** Find the objects that have J-H > 1.5

**R3:** Find the objects where Ks differs by more than 20 times the flux uncertainty from the mean flux.

**R4:** Find all catalogues that exist for a given field.

**R5:** For a given field I would like to retrieve the Y,Z,J,H and Ks magnitudes for all stars with S/N > 30 in Y, Z, J, H and Ks. 




In [87]:
# R1
con = lite.connect(root+'VVV.db')
R1command = "SELECT MJD, StarID, Filter FROM FieldTable \
            WHERE MJD BETWEEN 56800 AND 57300"
t_images = pd.read_sql(R1command, con)

"""for i, stars in enumerate(t_images['StarID']):
    # Need filter to get the right fluxes
    filt = t_images['Filter'][i]
    
    #Get the first and last starID"""

    
    

"for i, stars in enumerate(t_images['StarID']):\n    # Need filter to get the right fluxes\n    filt = t_images['Filter'][i]\n    \n    #Get the first and last starID"

In [104]:
# not right yet
t_images['StarID'][0][2:-1].split('    ')

[u'  0', u'1', u'2 ..., 9997 9998 9999']